In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
from sklearn.decomposition import PCA

In [6]:
import pandas as pd

In [7]:
def make_data_loaders(train_csv, val_csv, test_csv, images_dir, batch_size, image_size):
    from torch.utils.data import DataLoader, Dataset
    from PIL import Image

    class ImageDataset(Dataset):
        def __init__(self, csv_file, root_dir, transform=None):
            self.data = pd.read_csv(csv_file)
            self.root_dir = root_dir
            self.transform = transform

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
            image = Image.open(img_name).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image

    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = ImageDataset(train_csv, images_dir, transform)
    val_dataset = ImageDataset(val_csv, images_dir, transform)
    test_dataset = ImageDataset(test_csv, images_dir, transform)

    dataloaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
        'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False),
        'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    }
    return dataloaders

# Define the ResNet50 model with a custom embedding layer
class ResNet50Encoder(nn.Module):
    def __init__(self, embedding_dim=128):
        super(ResNet50Encoder, self).__init__()
        resnet = models.resnet50(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(resnet.fc.in_features, embedding_dim)
        self.normalize = nn.functional.normalize

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return self.normalize(x, p=2, dim=1)

# Define the MultiLabelTripletLoss
class MultiLabelTripletLoss(nn.Module):
    def __init__(self, margin=0.2):
        super(MultiLabelTripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        pos_dist = torch.sum((anchor - positive) ** 2, dim=1)
        neg_dist = torch.sum((anchor - negative) ** 2, dim=1)
        loss = torch.relu(pos_dist - neg_dist + self.margin)
        return torch.mean(loss)

# Clustering
from sklearn.cluster import AgglomerativeClustering

def perform_clustering(embeddings, n_clusters=14):
    clustering_model = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')
    cluster_labels = clustering_model.fit_predict(embeddings)
    return cluster_labels

# Training function
def train_model_unsupervised(model, dataloader, optimizer, criterion, device, num_epochs=10, n_clusters=14):
    model.train()
    for epoch in range(num_epochs):
        all_embeddings = []
        for imgs in dataloader:
            imgs = imgs.to(device)
            embeddings = model(imgs)
            all_embeddings.append(embeddings.detach().cpu().numpy())

        all_embeddings = np.vstack(all_embeddings)
        embeddings_np = all_embeddings.detach().cpu().numpy()
        pca = PCA(n_components=50)  # Reduce a 50 dimensiones
        reduced_embeddings = pca.fit_transform(embeddings_np)
        
        #cluster_labels = clustering_model.fit_predict(embeddings_np,n_clusters)
        cluster_labels = perform_clustering(reduced_embeddings, n_clusters=n_clusters)
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Cluster distribution: {np.bincount(cluster_labels)}")

        # Update triplets and train
        for imgs in dataloader:
            imgs = imgs.to(device)
            embeddings = model(imgs)
            # Here you would create triplets based on cluster_labels
            # For simplicity, assume we have anchor, positive, negative:
            anchor, positive, negative = embeddings[0], embeddings[1], embeddings[2]
            loss = criterion(anchor, positive, negative)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        torch.save(model.state_dict(), f"model_epoch_{epoch + 1}.pth")

In [8]:
train_csv = "C:/Users/RSCBAL04/Documents/GitHub/ChestX-ray8_classification/data/metadata/train.csv"
val_csv = "C:/Users/RSCBAL04/Documents/GitHub/ChestX-ray8_classification/data/metadata/train.csv"
test_csv = "C:/Users/RSCBAL04/Documents/GitHub/ChestX-ray8_classification/data/metadata/train.csv"
images_dir = "C:/Users/RSCBAL04/Documents/GitHub/ChestX-ray8_classification/data/images"
dataloaders = make_data_loaders(train_csv, val_csv, test_csv, images_dir, batch_size=32, image_size=224)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model and optimizer setup
model = ResNet50Encoder().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = MultiLabelTripletLoss()

# Train
dataloader = dataloaders['train']
train_model_unsupervised(model, dataloader, optimizer, criterion, device, num_epochs=10, n_clusters=14)


c:\Users\RSCBAL04\anaconda3\envs\marta_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\RSCBAL04\anaconda3\envs\marta_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.